## Load packages

In [1]:
# Useful packages
import os
import warnings

# Work with Python array and graphs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Cluster tendency
from pyclustertend import hopkins

# Mapping
from sklearn.metrics import pairwise_distances, accuracy_score
from coclust.evaluation.external import accuracy
from skbio.stats.ordination import pcoa
from multiview.mvmds import mvmds

# Clustering algorithm
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
#from KMedoidsPaper import KMedoids

# Clustering performance evaluation
from sklearn.metrics import davies_bouldin_score, silhouette_score

//anaconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/skbio/util/_testing.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as pdt


In [2]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

scikit-learn==0.22.1
numpy==1.17.0
matplotlib==3.2.1


## Load data

In [9]:
ISC_EDA = np.genfromtxt(os.path.join('school_EDA12.csv'),delimiter=',')
ISC_EDA = np.array(ISC_EDA,dtype='float') # Convert into array

ISC_IBI = np.genfromtxt(os.path.join('school_IBI.csv'),delimiter=',')
ISC_IBI = np.array(ISC_IBI,dtype='float') # Convert into array

allISC = [ISC_EDA,ISC_IBI]

nParticipants = ISC_EDA.shape[0]
nPeriod = int(np.sqrt(ISC_EDA.shape[1] / nParticipants))
ISC_EDA_sh = np.zeros((ISC_EDA.shape[0]*nPeriod,ISC_EDA.shape[0]*nPeriod))
nParticipants = ISC_IBI.shape[0]
nPeriod = int(np.sqrt(ISC_IBI.shape[1] / nParticipants))
ISC_IBI_sh = np.zeros((ISC_IBI.shape[0]*nPeriod,ISC_IBI.shape[0]*nPeriod))

allISC_sh = [ISC_EDA_sh,ISC_IBI_sh]

#ISC_IBI = ISC_IBI[0:10,0:10]
ISC_IBI = ISC_EDA

for i in range(len(allISC)):
    
    ISC = allISC[i]
    ISC_sh = allISC_sh[i]
    
    nParticipants = ISC.shape[0]
    nPeriod = int(np.sqrt(ISC.shape[1] / nParticipants))

    ISC_sh = np.zeros((ISC.shape[0]*nPeriod,ISC.shape[0]*nPeriod))

    for period1 in range(nPeriod):
        for period2 in range(nPeriod):
            if (period1 == period2):
                subISC_EDA = np.nan_to_num(ISC[:,nParticipants*(nPeriod*period1+period2):nParticipants*(nPeriod*period1+period2+1)])
                subISC_EDA = (subISC_EDA+subISC_EDA.T)/2
            else:
                subISC_EDA = ISC[:,nParticipants*(nPeriod*period1+period2):nParticipants*(nPeriod*period1+period2+1)]
            ISC_sh[nParticipants*period1:nParticipants*(period1+1),nParticipants*period2:nParticipants*(period2+1)] = subISC_EDA

    ISC_sh = np.nan_to_num(ISC_sh)
    ISC_sh = (ISC_sh+ISC_sh.T)/2

import csv
    
with open(os.path.join('test.csv'), 'w') as File:
    writer = csv.writer(File,delimiter =',')
    writer.writerows(ISC_EDA_sh)        
        
        

## Convert 2D to 4D

In [ ]:
'''
nPeriod = 7
ISC_EDA = np.zeros((ISC_EDA_2D.shape[0],int(ISC_EDA_2D.shape[1]/(nPeriod**2)),nPeriod,nPeriod))
for i in range(nPeriod):
    ISC_EDA[:,:,i] = ISC_EDA_2D[:,80*i:80*(i+1)]
    
print(ISC_EDA.shape)
'''

In [ ]:
'''nPeriod = 2
ISC_EDA = np.reshape(ISC_EDA,(ISC_EDA.shape[0],int(ISC_EDA.shape[1]/(nPeriod**2)),nPeriod,nPeriod))
ISC_IBI = np.reshape(ISC_IBI,(ISC_IBI.shape[0],int(ISC_IBI.shape[1]/(nPeriod**2)),nPeriod,nPeriod))

print(ISC_EDA.shape)
print(ISC_IBI.shape)
'''

## Pre processing specific to these matrices

In [ ]:
'''# Symetrical matrices
ISC_EDA = np.nan_to_num(ISC_EDA)
ISC_IBI = np.nan_to_num(ISC_IBI)
ISC_EDA = (ISC_EDA+np.swapaxes(ISC_EDA,0,1))/2
ISC_IBI = (ISC_IBI+np.swapaxes(ISC_IBI,0,1))/2

print(ISC_EDA[:4,:4,0,0])
print(ISC_IBI[:4,:4,0,0])

#ISC_EDA = ISC_EDA[:,:,0,0]
#ISC_IBI = ISC_IBI[:,:,0,0]
'''

In [ ]:
'''ISC_EDA = np.swapaxes(ISC_EDA,1,2)
ISC_EDA = np.flip(ISC_EDA,3)
print(ISC_EDA.shape[0])
ISC_EDA = np.reshape(ISC_EDA,((ISC_EDA.shape[0]*ISC_EDA.shape[1],ISC_EDA.shape[0]*ISC_EDA.shape[1])))
'''

In [50]:
def loadShapedMatrix(mat):
    ISC = np.genfromtxt(os.path.join('school_%s.csv' %mat),delimiter=',')
    ISC = np.array(ISC,dtype='float') # Convert into array

    nParticipants = ISC.shape[0]
    nPeriod = int(np.sqrt(ISC.shape[1] / nParticipants))
    ISC_sh = np.zeros((ISC.shape[0]*nPeriod,ISC.shape[0]*nPeriod))

    nParticipants = ISC.shape[0]
    nPeriod = int(np.sqrt(ISC.shape[1] / nParticipants))

    ISC_sh = np.zeros((ISC.shape[0]*nPeriod,ISC.shape[0]*nPeriod))

    for period1 in range(nPeriod):
        for period2 in range(nPeriod):
            if (period1 == period2):
                subISC_EDA = np.nan_to_num(ISC[:,nParticipants*(nPeriod*period1+period2):nParticipants*(nPeriod*period1+period2+1)])
                subISC_EDA = (subISC_EDA+subISC_EDA.T)/2
            else:
                subISC_EDA = ISC[:,nParticipants*(nPeriod*period1+period2):nParticipants*(nPeriod*period1+period2+1)]
            ISC_sh[nParticipants*period1:nParticipants*(period1+1),nParticipants*period2:nParticipants*(period2+1)] = subISC_EDA
    # print(ISC_EDA_sh)
    ISC_sh = np.nan_to_num(ISC_sh)
    ISC_sh = (ISC_sh+ISC_sh.T)/2

    return ISC_sh,nPeriod,nParticipants

ISC_EDA,nPeriod,nParticipants = loadShapedMatrix('EDA12')
#ISC_IBI,nPeriod,nParticipants = loadShapedMatrix('IBI12')
ISC_IBI = ISC_EDA


import csv
    
with open(os.path.join('test.csv'), 'w') as File:
    writer = csv.writer(File,delimiter =',')
    writer.writerows(ISC_EDA)        
    
        


# Remove rows with NaNs
def removeNaNSubject(matrix,subject):
    new_mat = np.copy(matrix)
    new_mat = np.delete(new_mat, (subject), axis=0)
    new_mat = np.delete(new_mat, (subject), axis=1)
    
    '''
    with open(os.path.join('school_EDA_truncated.csv'), 'w') as File:
        writer = csv.writer(File,delimiter =';')
        writer.writerows(matrix)
    '''
    
    return new_mat
# Remove outlier in vector if we find one
def removeSubjectsVector(vector,subjects):
    new_vect = np.copy(vector)
    for subj in subjects:
        new_vect = np.delete(new_vect, (subj), axis=0)
    
    return new_vect

def removeNaNISC(ISC):

    last_i = 0
    i = 0
    subjectsRemoved_EDA = []
    remove = False
    nbTotal = len(ISC) 
    while (last_i < nbTotal and i < nbTotal - 1):
        #print(nbTotal)
        for i in range(nbTotal):
            remove = False
            #print(i)
            for p in range(nPeriod):
                if ISC[p*nParticipants,i] == 0.:
                    print(p,nParticipants,i)
                    remove = True
                    break

            if remove:        
                ISC = removeNaNSubject(ISC,i)
                last_i = i
                nbTotal -= 1
                #subjectsRemoved_EDA.append(i - nbTotal + 84)
                subjectsRemoved_EDA.append(i)
                break
                
    print(subjectsRemoved_EDA)
    print(ISC)
    return ISC

    #print(last_i)

ISC_EDA = removeNaNISC(ISC_EDA)
    
import csv
    
with open(os.path.join('test.csv'), 'w') as File:
    writer = csv.writer(File,delimiter =',')
    writer.writerows(ISC_EDA)        
        


1 12 4
1 12 15
[4, 15]
[[ 1.00000000e+00  6.37051073e-04 -2.61231164e-03  5.29814469e-02
   6.65845507e-02  1.98344924e-02  3.99448108e-02 -2.81809301e-02
   2.74517993e-03 -5.10090644e-04 -1.55923997e-02  3.65620732e-02
   4.90872538e-03  2.49558439e-02  4.48140993e-02  2.43796086e-02
   5.06812548e-02  1.64625992e-02  2.66983402e-02 -3.57981629e-03
   9.03370597e-03  2.07368315e-02]
 [ 6.37051073e-04  1.00000000e+00  4.28282390e-02  2.94378457e-02
  -7.29108494e-03  2.08548494e-02  2.15766473e-02  1.22437371e-02
   2.37337624e-02 -2.23515011e-02  2.83020822e-02  2.00132953e-02
   2.25733339e-02  4.65858224e-02  2.72202158e-02  1.75468155e-02
   4.59557816e-02  4.18915226e-02  5.07778813e-02 -1.00154837e-04
   5.31188130e-03  2.91983717e-02]
 [-2.61231164e-03  4.28282390e-02  1.00000000e+00  1.44376243e-02
   1.59513263e-02  2.71206188e-02  4.08477407e-02  4.74072908e-02
   2.32658917e-02  4.25095435e-03  3.78597466e-02  3.82443523e-03
  -6.39143054e-03  2.02213082e-02  7.85733397e-03

In [ ]:
           

ISC_EDA = removeNanISC(ISC_EDA)
ISC_IBI = removeNanISC(ISC_IBI)
ISC_EDA = np.genfromtxt(os.path.join('school_EDA12.csv'),delimiter=',')
ISC_EDA = np.array(ISC_EDA,dtype='float') # Convert into array

ISC_IBI = np.genfromtxt(os.path.join('school_IBI.csv'),delimiter=',')
ISC_IBI = np.array(ISC_IBI,dtype='float') # Convert into array

allISC = [ISC_EDA,ISC_IBI]

nParticipants = ISC_EDA.shape[0]
nPeriod = int(np.sqrt(ISC_EDA.shape[1] / nParticipants))
ISC_EDA_sh = np.zeros((ISC_EDA.shape[0]*nPeriod,ISC_EDA.shape[0]*nPeriod))
nParticipants = ISC_IBI.shape[0]
nPeriod = int(np.sqrt(ISC_IBI.shape[1] / nParticipants))
ISC_IBI_sh = np.zeros((ISC_IBI.shape[0]*nPeriod,ISC_IBI.shape[0]*nPeriod))

allISC_sh = [ISC_EDA_sh,ISC_IBI_sh]

#ISC_IBI = ISC_IBI[0:10,0:10]
ISC_IBI = ISC_EDA

for i in range(len(allISC)):
    
    ISC = allISC[i]
    ISC_sh = allISC_sh[i]
    
    nParticipants = ISC.shape[0]
    nPeriod = int(np.sqrt(ISC.shape[1] / nParticipants))

    ISC_sh = np.zeros((ISC.shape[0]*nPeriod,ISC.shape[0]*nPeriod))

    for period1 in range(nPeriod):
        for period2 in range(nPeriod):
            if (period1 == period2):
                subISC_EDA = np.nan_to_num(ISC[:,nParticipants*(nPeriod*period1+period2):nParticipants*(nPeriod*period1+period2+1)])
                subISC_EDA = (subISC_EDA+subISC_EDA.T)/2
            else:
                subISC_EDA = ISC[:,nParticipants*(nPeriod*period1+period2):nParticipants*(nPeriod*period1+period2+1)]
            ISC_sh[nParticipants*period1:nParticipants*(period1+1),nParticipants*period2:nParticipants*(period2+1)] = subISC_EDA

    ISC_sh = np.nan_to_num(ISC_sh)
    ISC_sh = (ISC_sh+ISC_sh.T)/2

import csv
    
with open(os.path.join('test.csv'), 'w') as File:
    writer = csv.writer(File,delimiter =',')
    writer.writerows(ISC_EDA)        
        
        
'''

for ISC in allISC_sh:
    last_i = 0
    i = 0
    subjectsRemoved_IBI = []
    nbTotal = len(ISC)
    while (last_i < nbTotal and i < nbTotal - 1):
        #print(nbTotal)
        for i in range(nbTotal):
            #print(i)
            if ISC[0,i] == 0.:
                ISC = removeNaNSubject(ISC,i)
                last_i = i
                nbTotal -= 1
                #subjectsRemoved_IBI.append(i - nbTotal + 84)
                subjectsRemoved_IBI.append(i)
                break

    print(subjectsRemoved_IBI)
    '''

In [36]:
ISC = ISC_EDA
nParticipants = ISC.shape[0]
nPeriod = int(np.sqrt(ISC.shape[1] / nParticipants))
nPeriod
ISC_EDA.shape

(24, 24)

In [45]:
ISC_EDA = np.genfromtxt(os.path.join('school_EDA12.csv'),delimiter=',')
ISC_EDA = np.array(ISC_EDA,dtype='float') # Convert into array

ISC_IBI = np.genfromtxt(os.path.join('school_IBI.csv'),delimiter=',')
ISC_IBI = np.array(ISC_IBI,dtype='float') # Convert into array

allISC = [ISC_EDA,ISC_IBI]

nParticipants = ISC_EDA.shape[0]
nPeriod = int(np.sqrt(ISC_EDA.shape[1] / nParticipants))
ISC_EDA_sh = np.zeros((ISC_EDA.shape[0]*nPeriod,ISC_EDA.shape[0]*nPeriod))
nParticipants = ISC_IBI.shape[0]
nPeriod = int(np.sqrt(ISC_IBI.shape[1] / nParticipants))
ISC_IBI_sh = np.zeros((ISC_IBI.shape[0]*nPeriod,ISC_IBI.shape[0]*nPeriod))

allISC_sh = [ISC_EDA_sh,ISC_IBI_sh]

#ISC_IBI = ISC_IBI[0:10,0:10]
ISC_IBI = ISC_EDA

for i in range(len(allISC)):
    
    ISC = allISC[i]
    ISC_sh = allISC_sh[i]
    
    nParticipants = ISC.shape[0]
    nPeriod = int(np.sqrt(ISC.shape[1] / nParticipants))

    ISC_sh = np.zeros((ISC.shape[0]*nPeriod,ISC.shape[0]*nPeriod))

    for period1 in range(nPeriod):
        for period2 in range(nPeriod):
            if (period1 == period2):
                subISC_EDA = np.nan_to_num(ISC[:,nParticipants*(nPeriod*period1+period2):nParticipants*(nPeriod*period1+period2+1)])
                subISC_EDA = (subISC_EDA+subISC_EDA.T)/2
            else:
                subISC_EDA = ISC[:,nParticipants*(nPeriod*period1+period2):nParticipants*(nPeriod*period1+period2+1)]
            ISC_sh[nParticipants*period1:nParticipants*(period1+1),nParticipants*period2:nParticipants*(period2+1)] = subISC_EDA
   # print(ISC_EDA_sh)
    ISC_sh = np.nan_to_num(ISC_sh)
    ISC_sh = (ISC_sh+ISC_sh.T)/2

import csv
    
with open(os.path.join('test.csv'), 'w') as File:
    writer = csv.writer(File,delimiter =',')
    writer.writerows(ISC_EDA_sh)        
    
        


# Remove rows with NaNs
def removeNaNSubject(matrix,subject):
    new_mat = np.copy(matrix)
    new_mat = np.delete(new_mat, (subject), axis=0)
    new_mat = np.delete(new_mat, (subject), axis=1)
    
    '''
    with open(os.path.join('school_EDA_truncated.csv'), 'w') as File:
        writer = csv.writer(File,delimiter =';')
        writer.writerows(matrix)
    '''
    
    return new_mat
# Remove outlier in vector if we find one
def removeSubjectsVector(vector,subjects):
    new_vect = np.copy(vector)
    for subj in subjects:
        new_vect = np.delete(new_vect, (subj), axis=0)
    
    return new_vect

for ISC in allISC_sh:
    print(ISC)
    last_i = 0
    i = 0
    subjectsRemoved_EDA = []
    remove = False
    nbTotal = len(ISC) 
    while (last_i < nbTotal and i < nbTotal - 1):
        #print(nbTotal)
        for i in range(nbTotal):
            remove = False
            #print(i)
            for p in range(nPeriod):
                if ISC[p*nParticipants,i] == 0.:
                    print(p,nParticipants,i)
                    remove = True
                    break

            if remove:        
                ISC = removeNaNSubject(ISC,i)
                last_i = i
                nbTotal -= 1
                #subjectsRemoved_EDA.append(i - nbTotal + 84)
                subjectsRemoved_EDA.append(i)
                break
                
    print(subjectsRemoved_EDA)
    print(ISC)

    #print(last_i)
    
#print(ISC_EDA[:4,:4])
'''

for ISC in allISC_sh:
    last_i = 0
    i = 0
    subjectsRemoved_IBI = []
    nbTotal = len(ISC)
    while (last_i < nbTotal and i < nbTotal - 1):
        #print(nbTotal)
        for i in range(nbTotal):
            #print(i)
            if ISC[0,i] == 0.:
                ISC = removeNaNSubject(ISC,i)
                last_i = i
                nbTotal -= 1
                #subjectsRemoved_IBI.append(i - nbTotal + 84)
                subjectsRemoved_IBI.append(i)
                break

    print(subjectsRemoved_IBI)
    '''

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.

'\n\nfor ISC in allISC_sh:\n    last_i = 0\n    i = 0\n    subjectsRemoved_IBI = []\n    nbTotal = len(ISC)\n    while (last_i < nbTotal and i < nbTotal - 1):\n        #print(nbTotal)\n        for i in range(nbTotal):\n            #print(i)\n            if ISC[0,i] == 0.:\n                ISC = removeNaNSubject(ISC,i)\n                last_i = i\n                nbTotal -= 1\n                #subjectsRemoved_IBI.append(i - nbTotal + 84)\n                subjectsRemoved_IBI.append(i)\n                break\n\n    print(subjectsRemoved_IBI)\n    '

## Compute distance matrix

In [ ]:
def computeDistanceMatrix(study_matrix):
    
    # Normalise maximum value to 1
    normed_matrix = np.copy(study_matrix)
    normed_matrix = normed_matrix / np.max(abs(normed_matrix))
    
    # Convert into distance matrix
    distance_matrix = np.sqrt((1-normed_matrix)) # Formula in Matlab and in Scikit to convert 
        
    # Interval MDS normalization to spread values between 0 and 1
    a = np.min(np.sort(distance_matrix,axis=0)[1,:])
    distance_matrix = distance_matrix - a
    distance_matrix = distance_matrix - np.diag(np.diag(distance_matrix))
    distance_matrix = distance_matrix / np.max(distance_matrix)
    
    # To make sure output is perfectly symmetrical
    return (distance_matrix+distance_matrix.T)/2

## Multiview mapping

In [ ]:
def computeAllDistanceMatrix(ISC_EDA,ISC_IBI):
    
    # Make copy of 2 matrices (for each modality)
    mat_EDA = np.copy(ISC_EDA)
    mat_IBI = np.copy(ISC_IBI)
    
    # Compute the 2 distance matrices (according to each modality)
    distance_matrix_EDA = computeDistanceMatrix(ISC_EDA)
    distance_matrix_IBI = computeDistanceMatrix(ISC_IBI)
  
    # Remove direct correlation influence and compute correlation distance matrices
    for i in range(len(mat_EDA)):
        mat_EDA[i,i] = 0
    for i in range(len(mat_IBI)):
        mat_IBI[i,i] = 0
    distance_matrix_corr_EDA = pairwise_distances(mat_EDA,metric='correlation')
    distance_matrix_corr_IBI = pairwise_distances(mat_IBI,metric='correlation')

    # Return distance matrices
    return distance_matrix_EDA, distance_matrix_IBI, distance_matrix_corr_EDA, distance_matrix_corr_IBI


In [ ]:
'''def computeAllDistanceMatrix(ISC_EDA,ISC_IBI):
    
    # Make copy of 2 matrices (for each modality)
    mat_EDA = np.copy(ISC_EDA)
    
    # Compute the 2 distance matrices (according to each modality)
    distance_matrix_EDA = computeDistanceMatrix(ISC_EDA)
  
    # Remove direct correlation influence and compute correlation distance matrices
    for i in range(len(ISC_EDA)):
        mat_EDA[i,i] = 0
    distance_matrix_corr_EDA = pairwise_distances(mat_EDA,metric='correlation')

    # Return distance matrices
    return distance_matrix_EDA, distance_matrix_corr_EDA
'''

In [ ]:
def computeCoordinate(mat=['EDA','IBI','corrEDA','corrIBI'],added=False):

    # Compute all distance matrices
    distance_matrix_EDA, distance_matrix_IBI, distance_matrix_corr_EDA, distance_matrix_corr_IBI = computeAllDistanceMatrix(ISC_EDA,ISC_IBI)    
    #distance_matrix_EDA, distance_matrix_corr_EDA = computeAllDistanceMatrix(ISC_EDA,ISC_IBI)    

    # Choose which matrices to include into the multiview
    multiviewMat = []
    if ('EDA' in mat):
        multiviewMat.append(distance_matrix_EDA)
        distance_matrix = distance_matrix_EDA
    if ('IBI' in mat):
        multiviewMat.append(distance_matrix_IBI)
        distance_matrix = distance_matrix_IBI
    if ('corrEDA' in mat):
        multiviewMat.append(distance_matrix_corr_EDA)
    if ('corrIBI' in mat):
        multiviewMat.append(distance_matrix_corr_IBI)
        
    # Compute mapping
    embeddingDim = dim # to have a 2D map
    
    if len(mat)==1:
        points = np.array(pcoa(distance_matrix, method='eigh', number_of_dimensions=embeddingDim).samples)
        print("pcoa proportion explained : %s " %np.sum(pcoa(distance_matrix, method='eigh', number_of_dimensions=embeddingDim).proportion_explained))
    else:
        points = mvmds(multiviewMat,len(mat)*[True],embeddingDim,added=added)
    
    return points

In [ ]:
import umap.umap_ as umap
from sklearn import preprocessing

def computeUMAPCoordinate(mat,sslLabel=None,n_neighbors=3):

    if (mat==['EDA']):
        study_matrix = ISC_EDA
    if (mat==['IBI']):
        study_matrix = ISC_IBI

    print(dim)
    print(type(dim))
    fitter = umap.UMAP(n_components=int(dim),n_neighbors=n_neighbors,metric='correlation',min_dist=0.0,init='spectral',target_weight=0.5,n_epochs=5000).fit(study_matrix,sslLabel)
    #points = preprocessing.scale(fitter.embedding_,axis=0)
    points = (fitter.embedding_ - np.mean(fitter.embedding_,axis=0)) / np.std(fitter.embedding_,axis=0)
    #print(np.std(points))
    #print(np.mean(points))
    return points

In [ ]:
from sklearn.cluster import SpectralClustering

def computeSCCoordinate(mat,gamma=1):
    
    distance_matrix_EDA, distance_matrix_IBI, distance_matrix_corr_EDA, distance_matrix_corr_IBI = computeAllDistanceMatrix(ISC_EDA,ISC_IBI)    
    #distance_matrix_EDA, distance_matrix_corr_EDA = computeAllDistanceMatrix(ISC_EDA,ISC_IBI)    

    if (mat==['EDA']):
        distance_matrix = distance_matrix_EDA
    if (mat==['IBI']):
        distance_matrix = distance_matrix_IBI
    
    
    clusteringDist = SpectralClustering(n_clusters=dim+1, n_init=10,affinity='precomputed')
    affinity_matrix = np.exp(-gamma * distance_matrix ** 2)
    clusteringDist.fit(affinity_matrix)
    return clusteringDist.map_[:,1:]

In [ ]:
from sklearn.cluster import SpectralClustering

def computeNoneCoordinate(mat):
    
    distance_matrix_EDA, distance_matrix_IBI, distance_matrix_corr_EDA, distance_matrix_corr_IBI = computeAllDistanceMatrix(ISC_EDA,ISC_IBI)    
    #distance_matrix_EDA, distance_matrix_corr_EDA = computeAllDistanceMatrix(ISC_EDA,ISC_IBI)    

    if (mat==['EDA']):
        distance_matrix = distance_matrix_EDA
    if (mat==['IBI']):
        distance_matrix = distance_matrix_IBI
    
    return distance_matrix

## Compute clustering

In [ ]:
class Results():

    def __init__(self,mat,mapping,algo):
        self.mat = mat
        self.mapping = mapping
        self.algo = algo
        
    def twoClustersMethodResult(self):
        print("Blind results\n")
        
        # Cluster tendency
        print("Hopkins test")
        self.hopkinsCoef()
        print("%f +- %f" %(self.hopkins[0],self.hopkins[1]))
        
        # Found clusters
        print("Found clusters ")
        print(self.label)
        
        # Clustering quality evaluation
        print("Silhouette coefficient : %0.3f" %self.silhouetteCoef())
        print("DB index : %f " %self.dbScore())
        
        # Comparing to ground truth (if known)
        if (os.path.isfile(os.path.join('conditionSchool_EDA.csv')) and os.path.isfile(os.path.join('conditionSchool_IBI.csv'))):
            global groundTruth
            if (groundTruth):
                print("\n\nComparing to ground truth\n")
                print("Accuracy : %f " %self.accuracy())
                #print("Well participants : [%s]" % ", ".join(map(str, self.wellClassified)))        
                print("Misclassified participants : [%s]" % ", ".join(map(str, self.misClassified)))        

    def applyMethod(self,method):
        
        # Mapping
        if (mapping == 'UMAP'):
            points = computeUMAPCoordinate(self.mat)
            #print(points)
        elif (mapping == 'MDS'):
            points = computeCoordinate(self.mat,added=False)
        elif (mapping == 'MDS_scale'):
            points = computeCoordinate(self.mat,added=True)
        elif (mapping == 'SC'):
            points = computeSCCoordinate(self.mat)
        elif (mapping == 'None'): # distance matrix based clustering algorithm
            print("dist meth")
            points = computeNoneCoordinate(self.mat)
            if (algo == 'Spectral Clustering distance'):
                points = np.exp(-method.gamma * points ** 2)
        
        # Clustering
        # if (meth.distance_matrix):
        if (algo == 'K-Medoids'):
            meth = method.fit(pairwise_distances(points))
        else:
            meth = method.fit(points)

        # Store found labels and points location
        if hasattr(meth, 'labels_'):
            label = meth.labels_
        else:
            label = meth.predict(points)
            
        self.label = label
        self.best_points = points
         
    def showResultMap(self):
        
        # Set new figure
        plt.figure()
        ax = plt.axes([0,0,1.2,1.2])
        ax.set_aspect(aspect='equal')
        
        # Annotate points
        if mat == ['EDA']:
            n = len(ISC_EDA)
        elif mat == ['IBI']:
            n = len(ISC_IBI)        
        for i in range(n):
            ax.annotate(i,(self.best_points[i,0],self.best_points[i,1]),xytext=(self.best_points[i,0]+(np.max(self.best_points[:,0])-np.min(self.best_points[:,0]))/50,self.best_points[i,1]))     
    
        # Show color according to trueDisplay bool
        global trueDisplay
        if (trueDisplay):
            # Scatter points
            for i in range(n_clusters):
                ax.scatter(self.best_points[self.trueGroups[i],0],self.best_points[self.trueGroups[i],1],s=145,label=str(i))
            
            # Add legend
            TP = mpatches.Patch(color='blue', label='NA')
            TN = mpatches.Patch(color='red', label='SSA')
            FP_FN = mpatches.Patch(color='black', label='misclassified')
            #plt.legend(handles=[TP,TN,FP_FN])
            plt.title("%s on [%s]" % (self.algo, ", ".join(map(str, self.mat))))
            #plt.axis('off')
            #axes = plt.gca()
            #axes.set_xlim([-0.4,0.4])
            #axes.set_ylim([-0.4,0.7])
            plt.savefig(os.path.join('figures','GT_%s.png' %self.mat),bbox_inches='tight')
        else:
            # Scatter points
            for i in range(n_clusters):
                ax.scatter(self.best_points[self.label == i,0],self.best_points[self.label == i,1],s=145,label=str(i))
            
            # Add legend
            group0 = mpatches.Patch(color='darkgreen', label='First group')
            group1 = mpatches.Patch(color='darkorange', label='Second group')
            #plt.legend(handles=[group0,group1])       
            plt.title("%s on [%s]" % (self.algo, ", ".join(map(str, self.mat))))
            #plt.axis('off')
            plt.savefig(os.path.join('figures','clustering_%s.png' %self.mat),bbox_inches='tight')

        
    def dbScore(self):
        # Compute DB-score (small means good clustering)
        if len(np.unique(self.label))==1:
            warnings.warn("Labels correspond to only 1 group")
            return -1
        elif len(np.unique(self.label))==len(ISC_EDA):
            warnings.warn("Each participant correspond to one label")
            return -1
        elif len(np.unique(self.label))==len(ISC_IBI):
            warnings.warn("Each participant correspond to one label")
            return -1
        else:
            return davies_bouldin_score(self.best_points,self.label)
        
    def hopkinsCoef(self):
        # Average on different random data generated in Hopkins
        H = []
        if mat == ['EDA']:
            n = len(ISC_EDA)
        elif mat == ['IBI']:
            n = len(ISC_IBI)
        for p in range(100):
            H.append(hopkins(self.best_points,n)) 
        self.hopkins = [1-np.mean(H),np.std(H)]
        
        # Raise a warning if Hopkins test <= 0.5
        if (self.hopkins[0]<=0.5):
            warnings.warn("Hopkins test <= 0.5 : data set does not have clustering tendency")
            
    def silhouetteCoef(self):
        # Compute Silhouette Coefficient based on distance matrix (the closer it is to 1, the better the clustering is)
        if len(np.unique(self.label))==1:
            warnings.warn("Labels correspond to only 1 group")
            return -1
        elif len(np.unique(self.label))==len(ISC_EDA):
            warnings.warn("Each participant correspond to one label")
            return -1
        elif len(np.unique(self.label))==len(ISC_IBI):
            warnings.warn("Each participant correspond to one label")
            return -1
        else:
            return silhouette_score(self.best_points, self.label, metric='euclidean')
        
    def accuracy(self):
        # Read true labels from CSV file
        condition = np.genfromtxt(os.path.join('conditionSchool_%s.csv' %mat[0]),delimiter=',')
        condition = np.array(condition,dtype='int') # Convert into array
        if mat == ['EDA']:
            condition = removeSubjectsVector(condition,subjectsRemoved_EDA)
        elif mat == ['IBI']:
            condition = removeSubjectsVector(condition,subjectsRemoved_IBI)
        self.trueGroups = []
        for i in range(n_clusters):
            self.trueGroups.append(np.where(condition == i)[0])
        
        # Find which participants are misclassified and compute accuracy
        if mat == ['EDA']:
            subjects = np.arange(len(ISC_EDA))
        elif mat == ['IBI']:
            subjects = np.arange(len(ISC_IBI))
        self.wellClassified = subjects[self.label==condition]
        self.misClassified = subjects[self.label!=condition]
        '''if (len(self.misClassified) > len(self.wellClassified)):
            self.wellClassified, self.misClassified = self.misClassified, self.wellClassified
        
        return len(self.wellClassified) / len(ISC_EDA)
        '''
        print(len(ISC_IBI))
        print(condition)
        print(self.label)
        return accuracy(condition,self.label)

In [ ]:
# Choose which matrix or matrices to study
mat = ['IBI']
#mat = ['EDA','IBI']

# Decide if compare to ground truth
groundTruth = True

# Display found clusters or true clusters with misclassified participants in black
trueDisplay = True

# Choose mapping method
#mapping = 'UMAP'
#mapping = 'MDS'
#mapping = 'MDS_scale'
mapping = 'SC'
#mapping = 'None'

# Choose clustering algorithm
algo = 'K-Means'
#algo = 'Spectral Clustering'
#algo = 'Hierarchical Clustering'
#algo = 'K-Medoids'

#algo = 'Spectral Clustering distance'
#algo = 'Hierarchical Clustering distance'

In [ ]:
# Compute chosen clustering
method_result = Results(mat=mat,mapping=mapping,algo=algo)
n_clusters = 2
dim = 2

if (algo == 'K-Means'):
    method = KMeans(n_clusters=n_clusters,n_init=100)
elif (algo == 'Spectral Clustering'):
    method = SpectralClustering(n_clusters=n_clusters,n_init=10,gamma=1)
elif (algo == 'K-Medoids'):
    method = KMedoids(n_clusters=n_clusters,tmax=1000)
elif (algo == 'Hierarchical Clustering'):
    method = AgglomerativeClustering(n_clusters=n_clusters,linkage="ward")
    
elif (algo == 'Spectral Clustering distance'):
    method = SpectralClustering(n_clusters=n_clusters,n_init=10,gamma=1,affinity='precomputed')
elif (algo == 'Hierarchical Clustering distance'):
    method = AgglomerativeClustering(n_clusters=n_clusters,linkage="single",affinity='precomputed')
#elif (algo == 'Hierarchical Clustering distance'): # ward doesnt work with precomputed matrix
#    method = AgglomerativeClustering(n_clusters=n_clusters,linkage="ward",affinity='precomputed')
        
method_result.applyMethod(method)

# Show results
method_result.twoClustersMethodResult()
method_result.showResultMap()

plt.figure()
groundTruth = False
trueDisplay = False
method_result.showResultMap()

from sklearn.metrics import homogeneity_score
condition = np.genfromtxt(os.path.join('conditionSchool_%s.csv' %mat[0]),delimiter=',')
condition = np.array(condition,dtype='int') # Convert into array
if mat == ['EDA']:
    condition = removeSubjectsVector(condition,subjectsRemoved_EDA)
elif mat == ['IBI']:
    condition = removeSubjectsVector(condition,subjectsRemoved_IBI)
print("homogeneity")
print(homogeneity_score(condition,method_result.label))

## Change dim

In [ ]:
# Compute chosen clustering
method_result = Results(mat=mat,mapping=mapping,algo=algo)
n_clusters = 17
accDim = []
allDim = np.arange(2,30)

if (algo == 'K-Means'):
    method = KMeans(n_clusters=n_clusters,n_init=100)
elif (algo == 'Spectral Clustering'):
    method = SpectralClustering(n_clusters=n_clusters,n_init=10,gamma=1)
elif (algo == 'K-Medoids'):
    method = KMedoids(n_clusters=n_clusters,tmax=1000)
elif (algo == 'Hierarchical Clustering'):
    method = AgglomerativeClustering(n_clusters=n_clusters,linkage="ward")

for dim in allDim:
    method_result.applyMethod(method)
    accDim.append(method_result.accuracy())

In [ ]:
plt.plot(allDim,accDim)
plt.xlabel("number of dimensions")
plt.legend()

#

#

#

#

#


## Test rendering with pandas

In [ ]:
break
import pandas as pd
from pandas_profiling import ProfileReport

df = pd.DataFrame(
    np.random.rand(100, 5),
    columns=["a", "b", "c", "d", "e"]
)
profile = ProfileReport(df, title='Pandas Profiling Report')
profile.to_widgets()

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[3, 3] = np.nan
df.iloc[0, 2] = np.nan


In [ ]:
df.style

In [ ]:
# Remove rows with NaNs
def removeNaNSubject(matrix,subject,period1,period2):
    new_mat = np.copy(matrix)
    new_mat = np.delete(new_mat[:,:,period1,period2], (subject), axis=0)
    new_mat = np.delete(new_mat[:,:,period1,period2], (subject), axis=1)
    
    '''
    with open(os.path.join('school_EDA_truncated.csv'), 'w') as File:
        writer = csv.writer(File,delimiter =';')
        writer.writerows(matrix)
    '''
    
    return new_mat
# Remove outlier in vector if we find one
def removeSubjectsVector(vector,subjects):
    new_vect = np.copy(vector)
    for subj in subjects:
        new_vect = np.delete(new_vect, (subj), axis=0)
    
    return new_vect
'''
last_i = 0
i = 0
subjectsRemoved_EDA = []
nbTotal = ISC_EDA.shape[0]
while (last_i < nbTotal and i < nbTotal - 1):
    #print(nbTotal)
    for i in range(nbTotal):
        #print(i)
        if ISC_EDA[0,i] == 0.:
            ISC_EDA = removeNaNSubject(ISC_EDA,i)
            last_i = i
            nbTotal -= 1
            #subjectsRemoved_EDA.append(i - nbTotal + 84)
            subjectsRemoved_EDA.append(i)
            break

    #print(last_i)
    
print(ISC_EDA[:4,:4])
print(subjectsRemoved_EDA)
'''

last_i = 0
i = 0
subjectsRemoved_IBI = []
nbTotal = ISC_IBI.shape[0]
print(ISC_IBI.shape)
while (last_i < nbTotal and i < nbTotal - 1):
    #print(nbTotal)
    for i in range(nbTotal):
        for period1 in range(nPeriod):
            for period2 in range(nPeriod):
                #print(i,period1,period2)
                if ISC_IBI[0,i,period1,period2] == 0.:
                    ISC_IBI = removeNaNSubject(ISC_IBI,i,period1,period2)
                    last_i = i
                    nbTotal -= 1
                    #subjectsRemoved_IBI.append(i - nbTotal + 84)
                    subjectsRemoved_IBI.append(i)
                    break